In [3]:
# Contains all of the imports necessary for data exploration

#First pip install all libraries
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn

#Clone the repository
!git clone https://github.com/dregmi08/Milestone-2-Data-Exploration-Initial-Preprocessing.git

#Import all libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
fatal: destination path 'Milestone-2-Data-Exploration-Initial-Preprocessing' already exists and is not an empty directory.


In [4]:
# Import the Spotify dataset using pandas
spotify_df = pd.read_csv('Milestone-2-Data-Exploration-Initial-Preprocessing/reviews.csv')

# Printing shape of the spotify dataframe, this tells us that we have 61,594 observations and 5 features
print("Printing shape upon reading from the dataframe", spotify_df.shape)

# Printing what features we have in our dataset
print("Printing all the unique features:", spotify_df.columns)

# Print summary of data
print("Printing description about the numerical features:", spotify_df.describe())

# See if we have any null values/missing data 
# First see which columns have any null values
print(spotify_df.isnull().any())

# Since we see that the reply column does have null data, let's see how many null values exist in the reply column
print("Printing number of null entries for the Reply feature:", spotify_df['Reply'].isnull().sum())

# We have a ton of null data in the reply category (61378/61594 of the observations have a null value for the reply)
# We are dropping the reply feature from the dataset for the following reasons:
# 
# 1. Limited Contribution: Only 216 of 61,594 observations contain non-null values for this feature, meaning 99.6% of observations are null. 
#    With such a high degree of missingness, this feature is unlikely to add significant value to our model, and removing it will streamline our dataset.
# 
# 2. Textual Data and Noise Risk: Filling in missing data for 61k+ observations would introduce excessive noise, particularly because the feature 
#    contains text, which is challenging to impute meaningfully without bias.
# 
# 3. Low Relevance: The replies largely consist of generic comments from the Spotify team asking for improvement ideas, information which is already 
#    covered in the original review feature. As a result, this feature is redundant and does not enhance the dataset.

# Drop the feature
spotify_df = spotify_df.drop(columns = ['Reply'])

# Sanity check number of features and shape after dropping
print("Columns after dropping reply column:", spotify_df.columns)
print("Shape of data frame after dropping reply column:", spotify_df.shape)

# We are also dropping the Time_submitted column of the dataset:
# 
# 1. The Time_submitted column does not contribute much to what we're trying to find. Our project focuses on classifying reviews in 
# categories: "Positive", "Negative"(we are definitely doing these categories), "Slightly Positive", "Slightly Negative", "Neutral"
# (we might also classify reviews into these categories) and ranking what features/problems can be improved about Spotify from most important
# (most frequently complained about) to least important. The time a review was submitted doesn't give us any relevant information here,
# so we're going to drop it.
# 

# Drop the Time_submitted
spotify_df = spotify_df.drop(columns = ['Time_submitted'])

# Sanity check number of features and shape after droppping
print("Columns after dropping time submitted column:", spotify_df.columns)
print("Shape of data frame after dropping time submitted column:", spotify_df.shape)



Printing shape upon reading from the dataframe (61594, 5)
Printing all the unique features: Index(['Time_submitted', 'Review', 'Rating', 'Total_thumbsup', 'Reply'], dtype='object')
Printing description about the numerical features:              Rating  Total_thumbsup
count  61594.000000    61594.000000
mean       3.155989        7.649381
std        1.673285       89.323143
min        1.000000        0.000000
25%        1.000000        0.000000
50%        3.000000        0.000000
75%        5.000000        1.000000
max        5.000000     8195.000000
Time_submitted    False
Review            False
Rating            False
Total_thumbsup    False
Reply              True
dtype: bool
Printing number of null entries for the Reply feature: 61378
Columns after dropping reply column: Index(['Time_submitted', 'Review', 'Rating', 'Total_thumbsup'], dtype='object')
Shape of data frame after dropping reply column: (61594, 4)
Columns after dropping time submitted column: Index(['Review', 'Rating', '

In [5]:
spotify_df['Review_length'] = spotify_df['Review'].str.len()
spotify_df['Review_word_count'] = spotify_df['Review'].str.split().str.len()
spotify_df['Review_num_capitals'] = spotify_df['Review'].str.count(r'[A-Z]')
spotify_df.describe()

,Rating,Total_thumbsup,Review_length,Review_word_count,Review_num_capitals
count,61594.000000,61594.000000,61594.000000,61594.000000,61594.000000
mean,3.155989,7.649381,163.323457,30.886223,5.004871
std,1.673285,89.323143,119.940997,22.743188,10.716296
min,1.000000,0.000000,10.000000,2.000000,0.000000
25%,1.000000,0.000000,72.000000,14.000000,2.000000
50%,3.000000,0.000000,130.000000,25.000000,3.000000
75%,5.000000,1.000000,221.000000,42.000000,5.000000
max,5.000000,8195.000000,3753.000000,699.000000,398.000000
